# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df=pd.read_csv('insurance.csv')
print(df.head())
df.info()
df.describe()

   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


В данных пропусков нет, типы данных преобразовывать не нужно

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** не изменится

**Обоснование:** 
учитывая: 
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
(AB)^{T} = B^{T} A^{T}
$$

Пускай $Z$ - преобразованная матрица.
$$
Z = XP
$$
$$
a_1 = Zw_1
$$
$$
w_1 = (Z^T Z)^{-1} Z^T y = ((XP)^T(XP))^{-1}  (XP)^Ty = P^{-1}((XP)^{T}X)^{-1}  (XP)^Ty = P^{-1}((XP)^{T}X)^{-1}  P^TX^Ty = P^{-1}(P^TX^TX)^{-1}  P^TX^Ty = P^{-1}(X^TX)^{-1}(P^T)^{-1}  P^TX^Ty = P^{-1}(X^TX)^{-1}  X^Ty = P^{-1} w
$$
$$
a_1 = Zw_1 = XP P^{-1} w = Xw = a
$$

## Алгоритм преобразования

**Алгоритм**

1. Вызываю случайную матрицу (по размерам матрицы признаков)
2. Проверяю, действительно ли она обратимая
3. Умножаю изначальные признаки на случайную матрицу
4. Сравниваю r2 начальной матрицы и преобразованной

**Обоснование**

1. Случайная матрица должна быть квадратной, по количеству признаков (4*4)

## Проверка алгоритма

Сначала обучаю модень на изначальных данных и нахожу r2

In [3]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
r2 = r2_score(target_valid, predictions)
print(r2)

0.435227571270266


Беру рандомную матрицу и пробую сделать из нее обратную. Если ошибки нет - умножаю на исходную и повторяю все как на исходных данных.

In [4]:
features_matrix = features.values
random_matrix = np.random.randint(1, 10, (features_train.shape[1], features_train.shape[1]))
random_matrix_err = np.linalg.inv(random_matrix)
features_new = features_matrix @ random_matrix
features_train, features_valid, target_train, target_valid = train_test_split(
    features_new, target, test_size=0.25, random_state=12345)
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
r2 = r2_score(target_valid, predictions)
print(r2)

0.43522757126972256


r2 одинаковый на начальной матрице и после преобразования.